# The Basic Function

In [ ]:
def doubler(number):
    return number * 2

print(doubler(5))

# Function are Objects Too

Functions are known as "first class objects" in Python. This means they can be passed around just like any other data type. Because everything in Python is an object, you can also learn a lot about your functions.

In [ ]:
def doubler(number):
    return number * 2

print(doubler)
print(doubler(10))
print(doubler.__name__)
print(doubler.__doc__)


If the function doesn't have a docstring, then `__doc__` returns None. Let's fix that:

In [ ]:
def doubler(number):
        """Doubles the number passed to it"""
        return number * 2 
    
print(doubler.__doc__)

# Our First Decorator

**Definition:** *A decorator will take the function they are decorating and extend its behavior while not actually modifying what the function itself does.*

In [ ]:
def doubler(number):
    """Doubles the number passed to it"""
    return number * 2

def info(func):
    def wrapper(*args):
        print('Function name: ' + func.__name__)
        print('Function docstring: ' + str(func.__doc__))
        return func(*args)
    return wrapper

my_decorator = info(doubler)
print(my_decorator(2))

# Using Decorator Syntax

In [ ]:
@info
def doubler(number):
    """Doubles the number passed to it"""
    return number * 2

print(doubler(4))

# Stacking / Chaining Decorators

In [ ]:
def bold(func):
    def wrapper():
        return "<b>" + func() + "</b>"
    return wrapper

def italic(func):
    def wrapper():
        return "<i>" + func() + "</i>"
    return wrapper

@bold
@italic
def formatted_text():
    return 'Python rocks!'

print(formatted_text())

# Adding Arguments to Decorated Functions

Let's say we want to pass an argument to our `formatted_text` function to make it actually format text that is passed to it. How will that effect the decorator? Or will it?

In [ ]:
@bold
def formatted_text(text):
    return text

print(formatted_text('Python Rocks!'))

The `wrapper` function needs an argument passed to it. Let's try modifying that function and see what happens:

In [ ]:
def bold(func):
    def wrapper(*args):
        return "<b>" + func() + "</b>"
    return wrapper

@bold
def formatted_text(text):
    return text

print(formatted_text('Python Rocks!'))

Obviously we forgot something. But what?

In [ ]:
def bold(func):
    def wrapper(*args):                      # <-- We need *args passed to wrapper()
        return "<b>" + func(*args) + "</b>"  # <-- We need to add *args here too
    return wrapper

@bold
def formatted_text(text):
    return text

print(formatted_text('Python Rocks!'))

Now we have a fully functional decorator that allows us to pass arguments to the decorated function

## Exercise(s)

1) How do we add arguments to the decorator?

# Adding arguments to the decorator itself

In [ ]:
def my_decorator(arg1, arg2):
    print('Decorator arg1 = ' + str(arg1))
    print('Decorator arg2 = ' + str(arg2))

    def the_real_decorator(function):

        def wrapper(*args, **kwargs):
            print('Function {} args: {} kwargs: {}'.format(
                function.__name__, str(args), str(kwargs)))
            return function(*args, **kwargs)

        return wrapper

    return the_real_decorator

@my_decorator(3, 'Python')
def doubler(number):
    return number * 2

print(doubler(5))

# Decorator Obfuscation

In [ ]:
def bold(func):
    def wrapper(*args):
        return "<b>" + func(*args) + "</b>"
    return wrapper

@bold
def formatted_text(text):
    """
    Format the passed in text
    """
    return text

print(formatted_text.__name__)
print(formatted_text.__doc__)

Python provides a way to fix this in its `functools` module:

In [ ]:
from functools import wraps

def bold(func):
    @wraps(func)
    def wrapper(*args):
        return "<b>" + func(*args) + "</b>"
    return wrapper

@bold
def formatted_text(text):
    """
    Format the passed in text
    """
    return text

print(formatted_text.__name__)
print(formatted_text.__doc__)

# Practical Decorators

You can use decorators for many things. Some of the most popular uses including creating decorators for authentication (django / flask) and logging.

## A Logging Decorator

In [ ]:
import logging

def log(func):
    """
    Log what function is called
    """
    def wrap_log(*args, **kwargs):
        name = func.__name__
        logger = logging.getLogger(name)
        logger.setLevel(logging.INFO)

        # add file handler
        fh = logging.FileHandler("/home/mdriscoll/Desktop/%s.log" % name)
        fmt = '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
        formatter = logging.Formatter(fmt)
        fh.setFormatter(formatter)
        logger.addHandler(fh)

        logger.info("Running function: %s" % name)
        result = func(*args, **kwargs)
        logger.info("Result: %s" % result)
        return func
    return wrap_log


@log
def doubler(number):
    """Doubles the number passed to it"""
    return number * 2


## Exercise(s)

1) How can we tell our logger decorator where to log? 
2) Can you figure out how to make the logger decorator log to stdout?


## An Example from Flask

http://flask.pocoo.org/docs/0.12/quickstart/#a-minimal-application

In [ ]:
from flask import Flask
app = Flask(__name__)

@app.route('/')
def hello_world():
    return 'Hello, World!'